In [ ]:
# |default_exp mongo

# DB variables

In [ ]:
# |export

# to fix the forward reference issue by from_oid, get_entries in BaseEntity
from __future__ import annotations

from dotenv import load_dotenv, find_dotenv
import os


def load_env():
    _ = load_dotenv(find_dotenv())


def get_DB_VARS():
    load_env()
    DB_HOST = os.getenv("DB_HOST")
    DB_NAME = os.getenv("DB_NAME")
    DB_LOGS = os.getenv("DB_LOGS")
    return DB_HOST, DB_NAME, DB_LOGS

# get collection

In [ ]:
# |export

import pymongo


def get_collection(
    target_db="localhost", client_name="masque_et_la_plume", collection_name="episodes"
):
    """
    target_db can be 'localhost' or 'nas923'
    """
    client = pymongo.MongoClient(f"mongodb://{target_db}:27017/")
    db = client[client_name]
    collection = db[collection_name]
    return collection

# logs

In [ ]:
# |export

from datetime import datetime
import pymongo


def mongolog(operation: str, entite: str, desc: str):
    DB_HOST, DB_NAME, DB_LOGS = get_DB_VARS()
    if DB_LOGS in ["true", "True"]:
        coll_logs = get_collection(DB_HOST, DB_NAME, "logs")
        coll_logs.insert_one(
            {
                "operation": operation,
                "entite": entite,
                "desc": desc,
                "date": datetime.now(),
            }
        )


def print_logs(n: int = 10):
    """
    Print the last n logs
    """
    DB_HOST, DB_NAME, DB_LOGS = get_DB_VARS()
    coll_logs = get_collection(DB_HOST, DB_NAME, "logs")
    for i, log in enumerate(coll_logs.find().sort("date", pymongo.DESCENDING)):
        if i == n:
            break
        print(log)

In [ ]:
print_logs(5)

{'_id': ObjectId('679a68382b212aee21955fc2'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '16 nov. 1961 00:00 - Le Masque et les Goncourt : "L\'Etat sauvage" de Georges Conchon, 1964 -> 18 nov. 1964 00:00', 'date': datetime.datetime(2025, 1, 29, 18, 41, 12, 512000)}
{'_id': ObjectId('679a68382b212aee21955fbf'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '11 sept. 2016 00:00 - Le Masque et les Goncourt : "Chanson douce" de Leïla Slimani, 2016 -> 12 sept. 2016 00:00', 'date': datetime.datetime(2025, 1, 29, 18, 41, 12, 503000)}
{'_id': ObjectId('679a68382b212aee21955fbc'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '02 juil. 2017 00:00 - Le Masque et les Goncourt : "L\'Ordre du jour" d\'Eric Vuillard, 2017 -> 03 juil. 2017 00:00', 'date': datetime.datetime(2025, 1, 29, 18, 41, 12, 491000)}
{'_id': ObjectId('679a68382b212aee21955fb9'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '25 nov. 2018 00:00 - Le Masque et les Goncourt : 

# Base entity

c'est une classe de base issue de la refactorisation de Auteur et Editeur proposée par github copilot


> Étapes de Raisonnement pour la Factorisation
> Analyser les Classes Existantes :
> 
> Les classes Auteur et Editeur ont des méthodes similaires pour vérifier l'existence, insérer, mettre à jour et supprimer des documents dans la base de données MongoDB.
> Les méthodes exists, keep, remove, et get_oid sont communes aux deux classes.
> Identifier les Comportements Communs :
> 
> Les comportements communs peuvent être regroupés dans une classe de base.
> Les classes spécifiques (Auteur et Editeur) peuvent hériter de cette classe de base et ajouter des comportements spécifiques si nécessaire.
> Créer une Classe de Base :
> 
> Créer une classe de base BaseEntity qui contient les méthodes communes.
> La classe de base gérera les interactions avec la base de données.
> Hériter de la Classe de Base :
> 
> Les classes Auteur et Editeur hériteront de BaseEntity.
> Elles définiront uniquement les comportements spécifiques à chaque entité.



In [ ]:
# |export

from bson import ObjectId
from typing import List


class BaseEntity:
    def __init__(self, nom: str, collection_name: str):
        """
        BaseEntity is a class that represents a generic entity in the database.
        :param nom: The name of the entity.
        :param collection_name: The name of the collection.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        self.nom = nom

    def exists(self) -> bool:
        """
        Check if the entity exists in the database.
        :return: True if the entity exists, False otherwise.
        """
        return self.collection.find_one({"nom": self.nom}) is not None

    def keep(self):
        """
        Keep the entity in the database.
        """
        if not self.exists():
            mongolog("insert", self.collection.name, self.nom)
            self.collection.insert_one({"nom": self.nom})
        else:
            mongolog("update", self.collection.name, self.nom)

    def remove(self):
        """
        Remove the entity from the database.
        """
        self.collection.delete_one({"nom": self.nom})
        mongolog("delete", self.collection.name, self.nom)

    def get_oid(self) -> ObjectId:
        """
        Get the object id of the entity.
        :return: The object id of the entity. (bson.ObjectId)
        None if does not exist.
        """
        document = self.collection.find_one({"nom": self.nom})
        if document:
            return document["_id"]
        else:
            return None

    @classmethod
    def from_oid(cls, oid: ObjectId) -> cls:
        """
        Create an inherited BaseEntity from an oid of a mongo entry.
        :param oid: oid as ObjectId.
        :return: The inherited BaseEntity.
        """

        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=cls.collection
        )

        document = collection.find_one({"_id": oid})

        inst = cls(nom=document.get("nom"))
        return inst

    @classmethod
    def get_entries(cls, request="") -> List[cls]:
        """'
        retourne le resultat de la requete sous forme d'une liste d'instance de inherited BaseEntity
        par exemple : request="part of name"
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=cls.collection
        )
        query = {
            "nom": {
                "$regex": request,
                "$options": "i",  # "i" pour insensible à la casse
            }
        }
        result = collection.find(query)
        list_baseentity = [cls.from_oid(entry.get("_id")) for entry in result]
        return list_baseentity

    def __repr__(self):
        return self.nom

    def __str__(self):
        return self.nom

# Auteur

In [ ]:
# |export


class Auteur(BaseEntity):
    collection = "auteurs"

    def __init__(self, nom: str):
        """
        Auteur is a class that represents an author in the database auteurs.
        :param nom: The name of the author.
        """
        super().__init__(nom, self.collection)

In [ ]:
Auteur.get_entries("marcel")

[Marcel Proust]

In [ ]:
auteur1 = Auteur("Victor Hugo")
print(f"Est-ce que auteur1 existe ? {auteur1.exists()}")

auteur1.keep()
print(f"et maintenant, st-ce que auteur1 existe ? {auteur1.exists()}")
print(f"et voici l'id de auteur1 : {auteur1.get_oid()}")

auteur1.remove()
print(f"après nettoyage, est-ce que auteur1 existe ? {auteur1.exists()}")
print(f"et son oid : {auteur1.get_oid()}")

Est-ce que auteur1 existe ? False
et maintenant, st-ce que auteur1 existe ? True
et voici l'id de auteur1 : 67a521ad60b4498ed3f32622
après nettoyage, est-ce que auteur1 existe ? False
et son oid : None


In [ ]:
print_logs(5)

{'_id': ObjectId('67a521ad60b4498ed3f32624'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 6, 21, 55, 9, 78000)}
{'_id': ObjectId('67a521ad60b4498ed3f32621'), 'operation': 'insert', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 6, 21, 55, 9, 64000)}
{'_id': ObjectId('679a68382b212aee21955fc2'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '16 nov. 1961 00:00 - Le Masque et les Goncourt : "L\'Etat sauvage" de Georges Conchon, 1964 -> 18 nov. 1964 00:00', 'date': datetime.datetime(2025, 1, 29, 18, 41, 12, 512000)}
{'_id': ObjectId('679a68382b212aee21955fbf'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '11 sept. 2016 00:00 - Le Masque et les Goncourt : "Chanson douce" de Leïla Slimani, 2016 -> 12 sept. 2016 00:00', 'date': datetime.datetime(2025, 1, 29, 18, 41, 12, 503000)}
{'_id': ObjectId('679a68382b212aee21955fbc'), 'operation': 'force_update', 'entite': 'episodes', '

# Editeur

In [ ]:
# |export


class Editeur(BaseEntity):
    collection = "editeurs"

    def __init__(self, nom: str):
        """
        Editeur is a class that represents a publisher in the database editeurs.
        :param nom: The name of the publisher.
        """
        super().__init__(nom, self.collection)

In [ ]:
Editeur.get_entries()

[]

In [ ]:
editeur1 = Editeur("Gallimard")
print(f"Est-ce que editeur1 existe ? {editeur1.exists()}")

editeur1.keep()
print(f"et maintenant, st-ce que editeur1 existe ? {editeur1.exists()}")
print(f"et voici l'id de editeur1 : {editeur1.get_oid()}")

editeur1.remove()
print(f"après nettoyage, est-ce que editeur1 existe ? {editeur1.exists()}")
print(f"et son oid : {editeur1.get_oid()}")

Est-ce que editeur1 existe ? False
et maintenant, st-ce que editeur1 existe ? True
et voici l'id de editeur1 : 67a521e460b4498ed3f3262d
après nettoyage, est-ce que editeur1 existe ? False
et son oid : None


In [ ]:
print_logs(5)

{'_id': ObjectId('67a521e460b4498ed3f3262f'), 'operation': 'delete', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 2, 6, 21, 56, 4, 808000)}
{'_id': ObjectId('67a521e460b4498ed3f3262c'), 'operation': 'insert', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 2, 6, 21, 56, 4, 799000)}
{'_id': ObjectId('67a521ad60b4498ed3f32624'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 6, 21, 55, 9, 78000)}
{'_id': ObjectId('67a521ad60b4498ed3f32621'), 'operation': 'insert', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 6, 21, 55, 9, 64000)}
{'_id': ObjectId('679a68382b212aee21955fc2'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '16 nov. 1961 00:00 - Le Masque et les Goncourt : "L\'Etat sauvage" de Georges Conchon, 1964 -> 18 nov. 1964 00:00', 'date': datetime.datetime(2025, 1, 29, 18, 41, 12, 512000)}


# Critique

In [ ]:
# |export


class Critique(BaseEntity):
    collection = "critiques"

    def __init__(self, nom: str):
        """
        Critique is a class that represents a "critique" in the database critiques.
        :param nom: The name of the "critique".
        """
        super().__init__(nom, self.collection)

In [ ]:
Critique.get_entries()

[]

In [ ]:
critique1 = Critique("Patricia Martin")
print(f"Est-ce que critique1 existe ? {critique1.exists()}")

critique1.keep()
print(f"et maintenant, st-ce que critique1 existe ? {critique1.exists()}")
print(f"et voici l'id de critique1 : {critique1.get_oid()}")

critique1.remove()
print(f"après nettoyage, est-ce que critique1 existe ? {critique1.exists()}")
print(f"et son oid : {critique1.get_oid()}")

Est-ce que critique1 existe ? False
et maintenant, st-ce que critique1 existe ? True
et voici l'id de critique1 : 67a521ec60b4498ed3f32635
après nettoyage, est-ce que critique1 existe ? False
et son oid : None


In [ ]:
print_logs(5)

{'_id': ObjectId('67a521ec60b4498ed3f32637'), 'operation': 'delete', 'entite': 'critiques', 'desc': 'Patricia Martin', 'date': datetime.datetime(2025, 2, 6, 21, 56, 12, 93000)}
{'_id': ObjectId('67a521ec60b4498ed3f32634'), 'operation': 'insert', 'entite': 'critiques', 'desc': 'Patricia Martin', 'date': datetime.datetime(2025, 2, 6, 21, 56, 12, 83000)}
{'_id': ObjectId('67a521e460b4498ed3f3262f'), 'operation': 'delete', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 2, 6, 21, 56, 4, 808000)}
{'_id': ObjectId('67a521e460b4498ed3f3262c'), 'operation': 'insert', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 2, 6, 21, 56, 4, 799000)}
{'_id': ObjectId('67a521ad60b4498ed3f32624'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 2, 6, 21, 55, 9, 78000)}


# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("py mongo helper.ipynb", ".")